
# Jacobian for homogenous gas reactor
***Energy equation***
$$
\frac{dT}{dt}= -\frac{1}{\rho c_p}\sum_{k=1}^{N_{spec}}\dot{\omega_{k}} W_k h_k = S_T
$$
***Species equation***
$$
\frac{dY_k}{dt}=\frac{1}{\rho}\dot{\omega_{k}}W_k=S_{Y_k},\,\,\,k=1\ldots N_{spec}
$$


## Environment Setup

In [1]:
TChem_install_directory ='/Users/odiazib/Documents/CODE/TChem_CSPlib_github/install/tchem'

In [2]:
import sys
sys.path.append(TChem_install_directory+'/lib')
import pytchem
import numpy as np
import matplotlib.pyplot as plt
import time

## Pre-simulation
* Set variables; temperature and stoichiometric ratio (fuel/air).
* Convert from stoichiometric ratio to mass fraction of CH4, O2, N2 and AR.
* Create samples spaning over the variable ranges.

In [3]:
# Pressure, Temperature, and mass fractions
one_atm = 101325 # [Pa]
T = 1200 # K 
P = one_atm # Pa
Yp_fuel = 0.1
Yr_o2 = 0.9

In [4]:
N=2
Nvar = 4
sample = np.zeros([N,Nvar])

In [5]:
# sample 1
sample[0,0] = T
sample[0,1] = P
sample[0,2] = Yp_fuel
sample[0,3] = Yr_o2

# sample 2
sample[1,0] = T
sample[1,1] = P
sample[1,2] = Yp_fuel - 0.05
sample[1,3] = Yr_o2 + 0.05

## TChem Simulation

### Initialize TChem Driver Object

* Initialization of Kokkos.
* Create a TChem driver object. 

In [6]:
pytchem.initialize()

In [7]:
tchem = pytchem.TChemDriver()

* Get help from TChem driver object.

In [8]:
help(tchem)

Help on TChemDriver in module pytchem object:

class TChemDriver(pybind11_builtins.pybind11_object)
 |  A class to manage data movement between numpy to kokkos views in TChem::Driver object
 |  
 |  Method resolution order:
 |      TChemDriver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: pytchem.TChemDriver) -> None
 |  
 |  computeJacobianHomogeneousGasReactor(...)
 |      computeJacobianHomogeneousGasReactor(self: pytchem.TChemDriver) -> None
 |      
 |      Compute Jacobian matrix for homogeneous gas reactor
 |  
 |  computeNetProductionRatePerMass(...)
 |      computeNetProductionRatePerMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute net production rate
 |  
 |  computeRHS_HomogeneousGasReactor(...)
 |      computeRHS_HomogeneousGasReactor(self: pytchem.TChemDriver) -> None
 |      
 |      Compute RHS for homogeneous gas reactor
 |  
 |  computeTimeAdvanceHomogeneous

### Create Kinetic Model 

* Inputs are the reactions mechanism files; in this case, we use the GRI3.0 gas reaction mechanism

In [9]:
inputs_directory = TChem_install_directory + '/example/data/H2/'
tchem.createKineticModel(inputs_directory+'chem.inp',inputs_directory+'therm.dat')

### Set State Vector

* Get index for variables. 
* Pass a numpy array to the TChem object to set the state vector. 

In [10]:
Variables = ['T','P','H2','O2']
indx=[]
for var in Variables:
    indx += [tchem.getStateVariableIndex(var)]

state = np.zeros([N, tchem.getLengthOfStateVector()])
for sp in range(N):
    state[sp,indx] = sample[sp,:]

In [11]:
tchem.setNumberOfSamples(N)
tchem.createStateVector()
tchem.setStateVector(state)

In [12]:
tchem.computeJacobianHomogeneousGasReactor()

In [13]:
# get jacobian for sample 1
jacobian1 = tchem.getJacobianHomogeneousGasReactor(0)

In [14]:
print(jacobian1)

[[-2.58345151e+00  3.25805619e+02 -3.62006243e+01 -7.56177571e+08
   2.17834158e+10  2.22143362e+02 -2.73200920e+09 -2.54627165e+06
  -3.09404524e+07]
 [-5.79033697e-05 -1.07507891e-02 -2.10674121e-03 -4.94015786e+05
  -1.75716792e+06  2.12172226e-03  3.79213460e-02 -1.60025933e+02
   1.12373574e-03]
 [-9.19093759e-04 -1.70646045e-01 -3.34400793e-02  3.79213433e-02
  -7.62422727e-03  3.34842136e-02 -2.87152595e+07  6.63026992e+01
  -7.36375328e+00]
 [ 8.12846431e-13  3.52077758e-11  1.81936607e-11 -3.92072989e+06
   2.16490779e-02  7.28986233e-11  1.13232802e+07 -3.79251009e-12
  -3.68012539e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.16773778e+06
  -1.48242532e+07  1.02942976e-04  1.20366523e+07  0.00000000e+00
   1.23521650e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.57028372e+07 -1.09044062e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.89516923e-05  5.37539709e-03  1.05337056e-03  2.47007893e+05
   8.78583960e+05 -1.0608602

Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [15]:
# get jacobian for sample 2
jacobian2 = tchem.getJacobianHomogeneousGasReactor(1)

In [16]:
print(jacobian2)

[[-2.70755850e+00 -3.43636012e+02  1.80861059e+01 -7.50403380e+08
   2.16170795e+10  3.26815413e+02 -1.04729139e+10 -2.56275580e+06
  -3.70237761e+07]
 [-4.35936478e-05 -2.43746872e-02 -1.07173605e-03 -3.52353955e+05
  -1.25329005e+06  2.27864346e-03  4.07259730e-02 -1.14137171e+02
   1.20684618e-03]
 [-6.91957208e-04 -3.86896730e-01 -1.70115524e-02  4.07259716e-02
  -2.68833149e-02  3.58770479e-02 -4.19561129e+07  5.70079826e+01
  -1.10955849e+01]
 [ 7.37673301e-13  7.98247233e-11  1.48041190e-11 -2.79643834e+06
   3.25978214e-02  1.12766419e-10  1.70498934e+07 -4.07300071e-12
  -3.95230414e-12]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.97261532e+06
  -1.05733145e+07  1.55004544e-04  1.81240450e+07  0.00000000e+00
   1.06198297e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.11999595e+07 -1.64191145e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.17968271e-05  1.21873458e-02  5.35868004e-04  1.76176977e+05
   6.26645023e+05 -1.1393211

In [17]:
tchem.computeRHS_HomogeneousGasReactor()

In [18]:
tchem.getRHS_HomogeneousGasReactor(0)

array([-1.33285683e+02, -2.97114600e-03, -4.71606759e-02,  1.98950722e-11,
        0.00000000e+00,  0.00000000e+00,  1.48557322e-03,  4.86462486e-02,
        0.00000000e+00])

In [19]:
tchem.getRHS_HomogeneousGasReactor(1)

array([-1.39616127e+02, -2.23688361e-03, -3.55058113e-02,  1.80551492e-11,
        0.00000000e+00,  0.00000000e+00,  1.11844190e-03,  3.66242530e-02,
        0.00000000e+00])

Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [20]:
pytchem.finalize()